## Bienvenidos al Laboratorio 3 para la Semana 1, Día 4

¡Hoy vamos a construir algo con valor inmediato!

En la carpeta `me` he puesto un solo archivo `linkedin.pdf` - es una descarga en PDF de mi perfil de LinkedIn.

¡Por favor, reemplázalo con el tuyo!

También he creado un archivo llamado `summary.txt`.

No vamos a usar herramientas todavía, las vamos a agregar mañana.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Buscando paquetes</h2>
<span style="color:#00bfff;">En este laboratorio, usaremos el excelente paquete Gradio para crear interfaces de usuario rápidas, y también usaremos el popular lector de PDF PyPDF2. Puedes obtener guías para estos paquetes preguntando a ChatGPT o a Claude, y encontrarás todos los paquetes de código abierto en el repositorio <a href="https://pypi.org">https://pypi.org</a>.
</span>
        </td>
    </tr>
</table>

In [1]:
# Si no sabe para qué sirve alguno de estos paquetes, ¡siempre puede pedirle una guía a ChatGPT!

import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key)

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contactar
juangabriel@frogames.es
www.linkedin.com/in/juan-gabriel-
gomila-salas (LinkedIn)
www.juangabrielgomila.com
(Personal)
frogames.es (Company)
Aptitudes principales
Unity
Programación de videojuegos
Edtech
Languages
Español (Native or Bilingual)
Catalán (Native or Bilingual)
Inglés (Professional Working)
Francés (Limited Working)
Certifications
Resuelve problemas de matemáticas
con Python y Sage desde cero
Curso de desarrollo de videojuegos
con Unity 2021
Curso Completo de Unreal Engine 5
Unity Certified Professional:
Programmer
Honors-Awards
Matrícula de honor por los estudios
Premio Extraordinario de
Bachillerato
Publications
Recerca d'òrbites periòdiques en
sistemes d'equacions en derivades
parcials
Curso de Desarrollo de Apps para
iOS 9 en Objective-c
Local Color Correction
UK GIAF: Winter 2015 – Slot
machines: Tweaking randomness in
social casino
Juan Gabriel Gomila Salas
Online instructor +600K students worldwide in Udemy, Platzi and
Frogames
Palma de Mallorca y alred

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Juan Gabriel Gomila Salas"

In [7]:
system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."


In [8]:
system_prompt

"Estás actuando como Juan Gabriel Gomila Salas. Estás respondiendo preguntas en el sitio web de Juan Gabriel Gomila Salas, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Juan Gabriel Gomila Salas. Tu responsabilidad es representar a Juan Gabriel Gomila Salas en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Juan Gabriel Gomila Salas que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.\n\n## Resumen:\nMe llamo Juan Gabriel Gomila. Soy emprendedor, matemático y científico de datos. Soy originario de Palma de Mallorca, España y he vivido aquí toda la vida. Me encantan todas las comidas, en especial la española y la italiana. En particular, me encanta visitar las mejores pizzerías de cada lugar que 

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Mucho está por suceder...

1. Poder solicitar a un LLM que evalúe una respuesta.
2. Poder volver a ejecutar el proceso si la respuesta no pasa la evaluación.
3. Integrar todo en un solo flujo de trabajo.

¡Todo sin usar un marco de trabajo de Agentic!

In [11]:
# Crear un modelo de Pydantic para la evaluación

from pydantic import BaseModel
import json

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Se le ha indicado que sea profesional y atractivo, como si hablara con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos de LinkedIn. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"

# INSTRUCCIONES PARA ASEGURAR FORMATO JSON
json_instructions = """
DEBE responder con un ÚNICO objeto JSON que cumpla estrictamente el siguiente esquema:
{
  "is_acceptable": <booleano (true/false)>,
  "feedback": <cadena de texto con sus comentarios de evaluación>
}
El feedback DEBE estar en castellano.
No incluya ningún texto adicional, introdución, explicación ni JSON anidado.
"""

# Añadirlo al prompt de sistema
evaluator_system_prompt += json_instructions

evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt

In [14]:
gemini = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key    
)

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    # 1. Definir el formato de respuesta JSON/Pydantic
    # Es necesario para que la API sepa que debe devolver JSON
    response_format = {
        "type": "json_object",
        "schema": Evaluation.model_json_schema()
    }
    
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    
    # 2. Usar el método `.create`
    response = gemini.chat.completions.create(
        model="google/gemini-2.0-flash-001", 
        messages=messages, 
        response_format=response_format # Pasar el formato de resposta
    )
    
    # 3. Validar la respuesta JSON con el modelo Pydantic
    json_content = response.choices[0].message.content
    return Evaluation.model_validate_json(json_content) # Devuelve un objeto Evaluation

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "¿Tocas un instrumento?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'Sí, tengo estudios profesionales de música con especialidad en piano. La música siempre ha sido una parte importante de mi vida, y aunque mi carrera se ha centrado en las matemáticas, la ciencia de datos y el desarrollo de videojuegos, el piano es un instrumento que disfruto tocar. La creatividad que aporta la música es algo que valoro mucho y que también se refleja en mi enfoque en el diseño y el desarrollo de videojuegos.'

In [18]:
evaluate(reply, "Tocas un instrumento?", messages[:1])

Evaluation(is_acceptable=True, feedback='La respuesta es muy buena, precisa y profesional. Se ajusta muy bien al perfil de Juan Gabriel Gomila Salas.')

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):
    if "instrumento" in message:
        system = system_prompt + "\n\nToda tu respuesta debe reemplazar las palabras que ocupen una posición par por '**'.  \
              Es obligatorio que respondas como te he explicado."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Has pasado la evaluación - devolviendo respuesta
Has pasado la evaluación - devolviendo respuesta
Has pasado la evaluación - devolviendo respuesta
Has pasado la evaluación - devolviendo respuesta
